# ChronicCare AI Agent

_An Intelligent Health Assistant for Chronic Disease Monitoring_

## 1. Project Overview
ChronicCare AI is a virtual assistant designed to help users manage chronic diseases like diabetes, hypertension, and heart disease. It leverages AI to provide personalized recommendations related to lifestyle, diet, medications, and health alerts.

## 2. Goals of ChronicCare AI
- Monitor early symptoms of chronic conditions
- Suggest preventive care like medications, diet, and exercise
- Educate users about chronic diseases
- Recommend doctor visits when necessary
- Summarize previous user data and alert trends

## 3. Tools and Technologies Used
- **IBM Watsonx Assistant**: For building and deploying the chatbot
- **Python**: For backend data processing
- **NLU/NLP Models**: For symptom analysis and response generation
- **GitHub**: For version control and collaboration
- **Jupyter Notebook**: For documenting the workflow

## 4. Chatbot Interaction Flow
```mermaid
graph TD;
A[User Starts Chat] --> B[Greeting & Consent];
B --> C[Ask Age, Gender, Weight, Height];
C --> D[Ask Symptoms or Goals];
D --> E[Analyze Health Risk];
E --> F[Suggest Diet / Medicine / Exercise];
F --> G[Monitor Trends and Notify];
```

## 5. Sample Dialogs
**User:** Hi
**Agent:** Hello! Welcome to ChronicCare AI. Can I help you manage any chronic condition today?
**User:** I have high blood pressure.
**Agent:** Thanks for sharing. Please provide your age, weight, and recent symptoms.
**User:** 45 years old, 80 kg, headache and dizziness.
**Agent:** You may be experiencing hypertension symptoms. Please consult a physician. Meanwhile, reduce sodium, stay hydrated, and do light yoga daily.

## 6. Research Summary
- Analysis of common chronic diseases: diabetes, hypertension, cardiovascular
- Papers on diet and lifestyle-based prevention
- Existing chatbot frameworks and their limitations
- Research datasets used: MIMIC-III, UCI Diabetes Dataset

## 7. Future Enhancements
- Integration with wearable health devices
- Doctor recommendation system
- Emergency alert triggering
- More language support
- Support for more diseases like asthma, arthritis, and kidney disease

## 8. Screenshots and Demo Links
Include screenshots of IBM Watsonx Assistant, intents, entities, and chatbot output UI.


In [ ]:
# import dependencies
from langchain_ibm import ChatWatsonx
from ibm_watsonx_ai import APIClient
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from ibm_watsonx_ai.foundation_models.utils import Tool, Toolkit
import json
import requests

watsonx API connection

In [ ]:
his cell defines the credentials required to work with watsonx API for Foundation Model inferencing.

Action: Provide the IBM Cloud personal API key. For details, see documentation.

In [ ]:
import os
import getpass

def get_credentials():
	return {
		"url" : "https://us-south.ml.cloud.ibm.com",
		"apikey" : getpass.getpass("Please enter your api key (hit enter): ")
	}

def get_bearer_token():
    url = "https://iam.cloud.ibm.com/identity/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = f"grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey={credentials['apikey']}"

    response = requests.post(url, headers=headers, data=data)
    return response.json().get("access_token")

credentials = get_credentials()
     

Defining the model parameters

In [ ]:
We need to provide a set of model parameters that will influence the result:

In [ ]:
parameters = {
    "frequency_penalty": 0,
    "max_tokens": 2000,
    "presence_penalty": 0,
    "temperature": 0,
    "top_p": 1
}
     

Creating the agent

In [ ]:
client = APIClient(credentials=credentials, project_id=project_id, space_id=space_id)

# Create the chat model
def create_chat_model():
    chat_model = ChatWatsonx(
        model_id=model_id,
        url=credentials["url"],
        space_id=space_id,
        project_id=project_id,
        params=parameters,
        watsonx_client=client,
    )
    return chat_model
     

from ibm_watsonx_ai.deployments import RuntimeContext

context = RuntimeContext(api_client=client)




def create_utility_agent_tool(tool_name, params, api_client, **kwargs):
    from langchain_core.tools import StructuredTool
    utility_agent_tool = Toolkit(
        api_client=api_client
    ).get_tool(tool_name)

    tool_description = utility_agent_tool.get("description")

    if (kwargs.get("tool_description")):
        tool_description = kwargs.get("tool_description")
    elif (utility_agent_tool.get("agent_description")):
        tool_description = utility_agent_tool.get("agent_description")
    
    tool_schema = utility_agent_tool.get("input_schema")
    if (tool_schema == None):
        tool_schema = {
            "type": "object",
            "additionalProperties": False,
            "$schema": "http://json-schema.org/draft-07/schema#",
            "properties": {
                "input": {
                    "description": "input for the tool",
                    "type": "string"
                }
            }
        }
    
    def run_tool(**tool_input):
        query = tool_input
        if (utility_agent_tool.get("input_schema") == None):
            query = tool_input.get("input")

        results = utility_agent_tool.run(
            input=query,
            config=params
        )
        
        return results.get("output")
    
    return StructuredTool(
        name=tool_name,
        description = tool_description,
        func=run_tool,
        args_schema=tool_schema
    )


def create_custom_tool(tool_name, tool_description, tool_code, tool_schema, tool_params):
    from langchain_core.tools import StructuredTool
    import ast

    def call_tool(**kwargs):
        tree = ast.parse(tool_code, mode="exec")
        custom_tool_functions = [ x for x in tree.body if isinstance(x, ast.FunctionDef) ]
        function_name = custom_tool_functions[0].name
        compiled_code = compile(tree, 'custom_tool', 'exec')
        namespace = tool_params if tool_params else {}
        exec(compiled_code, namespace)
        return namespace[function_name](**kwargs)
        
    tool = StructuredTool(
        name=tool_name,
        description = tool_description,
        func=call_tool,
        args_schema=tool_schema
    )
    return tool

def create_custom_tools():
    custom_tools = []


def create_tools(context):
    tools = []
    
    config = {
    }
    tools.append(create_utility_agent_tool("Weather", config, client))
    config = {
    }
    tools.append(create_utility_agent_tool("WebCrawler", config, client))
    config = {
    }
    tools.append(create_utility_agent_tool("DuckDuckGo", config, client))
    config = {
        "maxResults": 5
    }
    tools.append(create_utility_agent_tool("Wikipedia", config, client))
    config = {
        "maxResults": 10
    }
    tools.append(create_utility_agent_tool("GoogleSearch", config, client))

    return tools
     